In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [3]:
#define tensors in pytorch - same as arrays in numpy - with tensor
X = torch.tensor(([2,9], [1,5], [3,6]), dtype=torch.float) #3x2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) #3 by 1 tensor
x_test = torch.tensor(([4, 8]))

#can check the size with .size()
X.size()

torch.Size([3, 2])

In [6]:
#apply scaling
#max function returns max element and indice
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(x_test, 0)

#divides all the elements by X_max - in this case X_max return largest single value in the matrices i.e. 9
X = torch.div(X, X_max)
x_test = torch.div(x_test, xPredicted_max)
y = y/100
print(X)

tensor([[0.6667, 1.0000],
        [0.3333, 0.5556],
        [1.0000, 0.6667]])


In [ ]:
#Building The Neural Network
#nn.module is the base class for any module in PyTorch
class NeuralNetwork(nn.Module):
    def __init__(self, x, y):
        super(NeuralNetwork, self).__init__()
        self.inputSize = X[0].size()
        self.outputSize = y[0].size()
        self.hiddenNodes = 3
        
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) #Tensor is 2 by 3 since (1 by 2) * (2 by 3) gives desired (1 by 3)
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) #Tesnsor is 3 by 1
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3)
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def backward(self, x, y, o):
        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        self.z2_error = torch.matmul
    